In [15]:
# 关联客户信息和产品信息
from imp4nb import *

cnx = lite.connect('data\\quandan.db')
# desclitedb(cnx)

cur = cnx.cursor()
result = cur.execute('select * from product')
colnames = [tuple1[0] for tuple1 in result.description]
print(colnames)
# print(result.fetchone()[0])

df = pd.read_sql_query(
    "select 日期,职员名称 as 业务主管,单据编号 as 销售单号,商品备注 as 订单单号,单据类型,摘要,xiaoshoumingxi.单位全名 as 客户,customer.往来单位编号 as 编码,"
    "xiaoshoumingxi.商品全名 as 商品全名,数量,xiaoshoumingxi.单价,金额,substr(customer.往来单位编号,1,2) as 大区 ,"
    "substr(customer.往来单位编号,1,3) as 小区 ,quyu.分部 as 分部,"
    "substr(customer.往来单位编号,12,1) as 类型,product.品牌名称  as 品牌 ,product.品牌分类  as 品牌分类 from xiaoshoumingxi,"
    "customer,product,quyu where (customer.往来单位 = xiaoshoumingxi.单位全名)  and (大区 = quyu.区域) "
    "and (product.商品全名 = xiaoshoumingxi.商品全名) order by 日期 desc", cnx,parse_dates=['日期'])
descdb(df)
cnx.close()

cnm = lite.connect(":memory:")
df['日期'] = df['日期'].apply(lambda x: pd.to_datetime(x))
df.to_sql('alldata',cnm)
df = pd.read_sql_query('select 日期,sum(金额) as 销售额 from alldata group by 日期',cnm)
descdb(df)

cnm.close()

['index', '商品全名', '条码', '分析编码', '品牌分类', '品牌名称', '重点品项', '推广等级', '单价']


1398531
                日期 业务主管                 销售单号  订单单号   单据类型  \
1398526 2012-11-30   黄琦  XT-2012-11-30-00009  None  销售退货单   
1398527 2012-11-30   黄琦  XT-2012-11-30-00009  None  销售退货单   
1398528 2012-11-30   黄琦  XT-2012-11-30-00009  None  销售退货单   
1398529 2012-11-30   黄琦  XT-2012-11-30-00009  None  销售退货单   
1398530 2012-11-30   黄琦  XT-2012-11-30-00009  None  销售退货单   

                                                      摘要  \
1398526  【联合一百湖工大学生超市 13207179589】销售退货【渔米之湘香酥麻辣鱼80g】等:黄琦   
1398527  【联合一百湖工大学生超市 13207179589】销售退货【渔米之湘香酥麻辣鱼80g】等:黄琦   
1398528  【联合一百湖工大学生超市 13207179589】销售退货【渔米之湘香酥麻辣鱼80g】等:黄琦   
1398529  【联合一百湖工大学生超市 13207179589】销售退货【渔米之湘香酥麻辣鱼80g】等:黄琦   
1398530  【联合一百湖工大学生超市 13207179589】销售退货【渔米之湘香酥麻辣鱼80g】等:黄琦   

                              客户               编码          商品全名   数量    单价  \
1398526  联合一百湖工大学生超市 13207179589  0810036HG01AXXC  渔米之湘广东盐焗鱼50g -2.0  2.30   
1398527  联合一百湖工大学生超市 13207179589  0810036HG01AXXC  创食人武汉黑鸭鸭脖33g -1.0  1.60   
1398528  联合一百湖工大学生超市 132071795

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398531 entries, 0 to 1398530
Data columns (total 18 columns):
日期      1398531 non-null datetime64[ns]
业务主管    1398531 non-null object
销售单号    1398531 non-null object
订单单号    1197044 non-null object
单据类型    1369782 non-null object
摘要      1368335 non-null object
客户      1398531 non-null object
编码      1398531 non-null object
商品全名    1398531 non-null object
数量      1398531 non-null float64
单价      1397657 non-null float64
金额      1397657 non-null float64
大区      1398531 non-null object
小区      1398531 non-null object
分部      1398531 non-null object
类型      1398531 non-null object
品牌      1398531 non-null object
品牌分类    1398531 non-null object
dtypes: datetime64[ns](1), float64(3), object(14)
memory usage: 192.1+ MB
None
(1398531, 18)


                 数量            单价            金额
count  1.398531e+06  1.397657e+06  1.397657e+06
mean   3.601970e+01  1.559901e+00  3.836879e+01
std    1.999320e+02  1.545035e+00  1.797522e+02
min   -4.860000e+03  0.000000e+00 -1.050000e+04
25%    1.000000e+01  7.500000e-01  1.500000e+01
50%    2.000000e+01  1.500000e+00  2.300000e+01
75%    3.000000e+01  1.800000e+00  3.750000e+01
max    2.800000e+04  8.800000e+01  3.500000e+04


1491
                       日期       销售额
1486  2017-10-18 00:00:00  15216.83
1487  2017-10-19 00:00:00  41283.50
1488  2017-10-20 00:00:00  50497.69
1489  2017-10-21 00:00:00  25043.00
1490  2017-10-23 00:00:00  54757.04
日期      object
销售额    float64
dtype: object
Index(['日期', '销售额'], dtype='object')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 2 columns):
日期     1491 non-null object
销售额    1491 non-null float64
dtypes: float64(1), object(1)
memory usage: 23.4+ KB
None
(1491, 2)
                 销售额
count    1491.000000
mean    35966.738638
std     19548.012352
min    -10500.000000
25%     24121.400000
50%     33557.880000
75%     45406.955000
max    154771.150000


In [2]:
from imp4nb import *

cnx = lite.connect('data\\quandan.db')

df = pd.read_sql_query('select * from xiaoshoumingxi',cnx)
# descdb(df)

dfcm = df.duplicated()
print(list(dfcm[dfcm == True].index))
df =df.drop_duplicates()
# descdb(df)

cnx.close()

[]
